# **MÓDULO 35 - Cross Validation**

Nesta tarefa, você trabalhará com uma base de dados que contém informações sobre variáveis ambientais coletadas para a detecção de incêndios. O objetivo é utilizar técnicas de validação cruzada (cross-validation) para avaliar a performance de um modelo de classificação na previsão da ocorrência de um incêndio com base nas variáveis fornecidas.


Descrição da Base de Dados
A base de dados contém as seguintes variáveis:

Unnamed:0: Índice (não é uma variável útil para o modelo)

UTC: Tempo em Segundos UTC

Temperature[C]: Temperatura do Ar (em graus Celsius)

Humidity[%]: Umidade do Ar (em porcentagem)

TVOC[ppb]: Total de Compostos Orgânicos Voláteis (medido em partes por bilhão)

eCO2[ppm]: Concentração equivalente de CO2 (medido em partes por milhão)

Raw H2: Hidrogênio molecular bruto, não compensado

Raw Ethanol: Etanol gasoso bruto

Pressure[hPA]: Pressão do Ar (em hectopascais)

PM1.0: Material particulado de tamanho < 1,0 µm

PM2.5: Material particulado de tamanho >1,0 µm e < 2,5 µm

NC0.5: Concentração numérica de material particulado de tamanho < 0,5 µm

NC1.0: Concentração numérica de material particulado de tamanho 0,5 µm < 1,0 µm

NC2.5: Concentração numérica de material particulado de tamanho 1,0 µm < 2,5 µm

CNT: Contador de amostras


E a variável alvo:

Fire Alarm: Indicador binário de incêndio (1 se houver incêndio, 0 caso contrário)

O objetivo desta tarefa é aplicar a técnica de validação cruzada (cross-validation) para avaliar a performance de um modelo de classificação. A validação cruzada ajudará a garantir que o modelo seja avaliado de maneira robusta e generalize bem para dados não vistos.

In [16]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# 1 - Carregue a base de dados, verifique os tipos de dados e também se há presença de dados faltantes ou nulos.

In [5]:
df = pd.read_csv("smoke_detection.csv", delimiter=',')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62630 entries, 0 to 62629
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      62630 non-null  int64  
 1   UTC             62630 non-null  int64  
 2   Temperature[C]  62630 non-null  float64
 3   Humidity[%]     62630 non-null  float64
 4   TVOC[ppb]       62630 non-null  int64  
 5   eCO2[ppm]       62630 non-null  int64  
 6   Raw H2          62630 non-null  int64  
 7   Raw Ethanol     62630 non-null  int64  
 8   Pressure[hPa]   62630 non-null  float64
 9   PM1.0           62630 non-null  float64
 10  PM2.5           62630 non-null  float64
 11  NC0.5           62630 non-null  float64
 12  NC1.0           62630 non-null  float64
 13  NC2.5           62630 non-null  float64
 14  CNT             62630 non-null  int64  
 15  Fire Alarm      62630 non-null  int64  
dtypes: float64(8), int64(8)
memory usage: 7.6 MB


In [7]:
df.rename(columns={'Fire Alarm': 'Fire_Alarm'}, inplace=True)

In [8]:
# verificação de valores vaziu ou nulo
total_nulos = df.isnull().sum().sum()
total_vazios = (df == '').sum().sum()

print(f"Total de valores nulos: {total_nulos}")
print(f"Total de campos vazios: {total_vazios}")

Total de valores nulos: 0
Total de campos vazios: 0


Para a coluna Fire Alarm, por conta do espaçamento talvez seja util renomear o nome da coluna utilizando:

df.rename(columns={'Fire Alarm': 'Fire_Alarm'}, inplace=True)

In [10]:
df['Fire_Alarm'].value_counts(), df['Fire_Alarm'].value_counts(normalize=True)

(Fire_Alarm
 1    44757
 0    17873
 Name: count, dtype: int64,
 Fire_Alarm
 1    0.714626
 0    0.285374
 Name: proportion, dtype: float64)

# 2 - Para essa base, onde você realizará as previsões de fire alarm, qual modelo de machine learning você aplicará? Justifique.




Apos o processamento, padronização e ajustes o que melhor vai se aplicar vai ser o random forest por ele vai se adptar bem.

# 3 - Separe a base em Y e X e já rode a instância do modelo que você utilizará.

In [12]:
# Separa X e y
X = df.drop(columns=['Fire_Alarm'])
y = df['Fire_Alarm']

# Divide em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.20,       # 20% para teste
    random_state=42,      # reprodutibilidade
    stratify=y            # mantém proporção da classe
)

print(X_train.shape, X_test.shape)
print(y_train.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))

(50104, 15) (12526, 15)
Fire_Alarm
1    0.714634
0    0.285366
Name: proportion, dtype: float64
Fire_Alarm
1    0.714594
0    0.285406
Name: proportion, dtype: float64


In [18]:
# usando SMOTE para balancer os dados de treino
sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)

In [19]:
# Aplicação do modelo Random Forest
rf = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    class_weight='balanced'
)

In [21]:
# Treinando o modelo
rf.fit(X_train_bal, y_train_bal)

RandomForestClassifier(class_weight='balanced', n_estimators=300,
                       random_state=42)

# 4 - Defina o número de Folds e rode o modelo com a validação cruzada.

In [23]:
folder = 5

# 5 - Avalie a pontuação de cada modelo e ao final a validação final da média.

In [24]:
crossvalidation = KFold(n_splits=folder, shuffle=True, random_state=5)

In [26]:
rf_final = cross_val_score(rf, X_train_bal, y_train_bal, cv=folder)

In [27]:
pontuação = cross_val_score(rf, X_train_bal, y_train_bal, cv=crossvalidation)

In [28]:
print(f"Pontuação de cada modelo: {pontuação}")

Pontuação de cada modelo: [1.         0.99986036 0.99979053 0.99993018 1.        ]


In [29]:
print((rf_final.mean()))

0.9998882886341136


Podemos verificar que o modelo se provou eficar usando os metodos de SMOTE e usando o random forest.
Os dois teste que foi aplicado acima se mostrou acima de 99%, tanto com a escolha padrão do modelo e com 5 folders.